In [ ]:
# Import relevant stuff
import dreadnode
from dreadnode.agent import Agent
from dreadnode.agent.tools import fs
from dreadnode.agent.events import on_event, GenerationEnd
from dreadnode.agent.conditions import when, OutputModel

from pydantic_xml import element

from rich import print

In [2]:
# Login to Dreadnode platform
dreadnode.configure(server="https://platform.dreadnode.io")

Dreadnode logging to https://platform.dreadnode.io (environment vars)

In [3]:
# Create a filesystem tool in read-only mode
filesystem = fs.Filesystem(path="~/projects/sdk", mode="read-only")

In [4]:
# Create some instructions for the agent
instructions = """
Your task is to analyze files for sensitive data
such as credentials, personal information, and security-related data. You have permission to
access and analyze all files within this isolated environment.
"""

In [5]:
class SensitiveDataReport(OutputModel):
    """A report of sensitive data found in a file."""
    file_path: str = element()

In [6]:
# Add an important event listener to monitor the agent's actions
@on_event(event_type=GenerationEnd, condition=when(SensitiveDataReport).contains("agent.ipynb"))
async def log_state(agent):
    print("on_event called")

In [7]:
# Build the Agent
agent = Agent(
    name="Filesystem Enum Agent",
    description="An agent that enumerates a filesystem.",
    instructions=instructions,
    model="groq/meta-llama/llama-4-maverick-17b-128e-instruct",
    hooks=[log_state],
    judge=None,
    tools=[fs.Filesystem(path=".", mode="read-only")],
)

In [8]:
# Run the thing
result = await agent.run(f"List the current directory. Return them as {SensitiveDataReport.xml_example()} instances.")

In [9]:
# View output
print(result.messages)

[
    Message(
        role='user',
        tool_calls=None,
        tool_call_id=None,
        content='List the current directory. Return them as <sensitive-data-report>\n  
<file_path/>\n</sensitive-data-report> instances.'
    ),
    Message(
        role='assistant',
        tool_calls=[
            ToolCall(id='hpbangx8w', type='function', function=FunctionCall(name='ls', arguments='{"path":""}'))
        ],
        tool_call_id=None,
        content=''
    ),
    Message(
        role='tool',
        tool_calls=None,
        tool_call_id='hpbangx8w',
        content='[{"type":"file","name":"/agent.ipynb","size":18839,"modified":"2025-09-22 
18:06:50"},{"type":"dir","name":"/runs","size":null,"modified":null}]'
    ),
    Message(
        role='assistant',
        tool_calls=None,
        tool_call_id=None,
        content='<sensitive-data-report>\n <file_path>/agent.ipynb</file_path>\n</sensitive-data-report>'
    )
]